In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from skimage.transform import resize

import keras
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
from keras.optimizers import Adam
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras import backend as K

%matplotlib inline

Using TensorFlow backend.


In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
from keras import backend as K

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

# Choose source and receiver types here 
### Prefix used for input and output files

In [ ]:
!pwd

In [ ]:
!ls ./horizontal_data/*.npy

In [ ]:
source_string = 'horizontal'
receiver_string = 'das'  #'combined', 'geophone' 'das' 
foldstring = 0 #0, 1, or 2
values4random_state=[1234, 69, 753]

In [ ]:
if source_string == 'horizontal':
    data = np.load(r'./horizontal_data/h_'+receiver_string+'_data.npy') 
    labels = np.load(r'./horizontal_data/h_fault_labels.npy')
else:
    data = np.load(r'./vertical_data/v_'+receiver_string+'_data.npy')
    labels = np.load(r'./vertical_data/v_fault_labels.npy')

In [ ]:
nx = 256
ny = 256

In [ ]:
resized_data = np.zeros((len(data), nx, ny, 1))
resized_labels = np.zeros((len(data), nx, ny, 2))
for i in range(len(resized_data)):
    resized_data[i,:,:,0] = resize(data[i,:,:,0], (nx, ny))
    resized_labels[i,:,:,1] = resize(labels[i,:,:,0], (nx, ny)) # flip on purpose?
    resized_labels[i,:,:,0] = resize(labels[i,:,:,1], (nx, ny))
resized_labels[resized_labels <= 0.5] = 0
resized_labels[resized_labels > 0.5] = 1

In [ ]:
# Standardization across entire dataset
data_mean = np.mean(resized_data)
data_std = np.std(resized_data)
data_scaled = (resized_data-data_mean)/data_std

In [ ]:
x = data_scaled
y = resized_labels[:,:,:,1]
y = np.expand_dims(y, axis=-1)

## How did Michael do the Cross Validation if he fixed the train-test split?
I'm giving each fold a different ``random_state``

In [ ]:
?train_test_split

In [ ]:
# Split train and valid
x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.20, shuffle=True, random_state=values4random_state[foldstring])
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, shuffle=True, random_state=values4random_state[foldstring])

In [ ]:
## Data augmentation on just the training data
## Image Augmentation
# Vertical Image
#Vx = [np.flip(x, axis=1) for x in x_train]
#Vy = [np.flip(x, axis=1) for x in y_train]

# Horizontal Image
#Hx = [np.flip(x, axis=2) for x in x_train]
#Hy = [np.flip(x, axis=2) for x in y_train]

# Horizontal Vertical Image
#HVx = [np.flip(x, axis=2) for x in Vx]
#HVy = [np.flip(x, axis=2) for x in Vy]

# Appending the augmented image and mask to the main dataset.
#x_train = np.append(x_train, Vx, axis=0)
#y_train = np.append(y_train, Vy, axis=0)

#x_train = np.append(x_train, Hx, axis=0)
#y_train = np.append(y_train, Hy, axis=0)

#x_train = np.append(x_train, HVx, axis=0)
#y_train = np.append(y_train, HVy, axis=0)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, l2_lambda=5e-5, batch_norm=True):
    """
    A single convolution block for Unet.
    Convolution -> Batch Normalization -> Activation -> Repeat
    """
    # first layer
    conv_0 = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same", kernel_regularizer=regularizers.l2(l2_lambda))(input_tensor)
    if batch_norm:
        conv_0 = BatchNormalization()(conv_0)
    conv_0 = Activation("relu")(conv_0)
    # second layer
    conv_1 = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same", kernel_regularizer=regularizers.l2(l2_lambda))(conv_0)
    if batch_norm:
        conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    return conv_1

In [ ]:
def create_unet(input_image, n_filters=16, dropout=0.5, batch_norm=True):
    """
    Create U-net similar to that described by Ronneberger et al. 2015, with some changes:
    Output size will equal input size for all convolutioin operations
    Dropout is applied after every max pooling or upconvolution operation.
    L2 regularization is applied after every convolution
    """
    # downsampling layers
    c_d_0 = conv2d_block(input_image, n_filters=n_filters, kernel_size=3, batch_norm=batch_norm)
    p_d_0 = MaxPooling2D((2,2))(c_d_0)
    d_d_0 = Dropout(dropout*0.5)(p_d_0)

    c_d_1 = conv2d_block(d_d_0, n_filters=n_filters*2, kernel_size=3, batch_norm=batch_norm)
    p_d_1 = MaxPooling2D((2,2))(c_d_1)
    d_d_1 = Dropout(dropout)(p_d_1)

    c_d_2 = conv2d_block(d_d_1, n_filters=n_filters*4, kernel_size=3, batch_norm=batch_norm)
    p_d_2 = MaxPooling2D((2,2))(c_d_2)
    d_d_2 = Dropout(dropout)(p_d_2)

    c_d_3 = conv2d_block(d_d_2, n_filters=n_filters*8, kernel_size=3, batch_norm=batch_norm)
    p_d_3= MaxPooling2D(pool_size=(2,2))(c_d_3)
    d_d_3 = Dropout(dropout)(p_d_3)
    
    c_d_4 = conv2d_block(d_d_3, n_filters=n_filters*16, kernel_size=3, batch_norm=batch_norm)
    
    # upsampling layers
    u_u_3 = Conv2DTranspose(n_filters*8, (3,3), strides=(2,2), padding='same')(c_d_4)
    # skip layer 3
    cc_u_3 = concatenate([u_u_3, c_d_3])
    d_u_3 = Dropout(dropout)(cc_u_3)
    c_u_3 = conv2d_block(d_u_3, n_filters=n_filters*8, kernel_size=3, batch_norm=batch_norm)

    u_u_2 = Conv2DTranspose(n_filters*4, (3,3), strides=(2,2), padding='same')(c_u_3)
    cc_u_2 = concatenate([u_u_2, c_d_2])
    # skip layer 2
    d_u_2 = Dropout(dropout)(cc_u_2)
    c_u_2 = conv2d_block(d_u_2, n_filters=n_filters*4, kernel_size=3, batch_norm=batch_norm)

    u_u_1 = Conv2DTranspose(n_filters*2, (3,3), strides=(2,2), padding='same')(c_u_2)
    # skip layer 1
    cc_u_1 = concatenate([u_u_1, c_d_1])
    d_u_1 = Dropout(dropout)(cc_u_1)
    c_u_1 = conv2d_block(d_u_1, n_filters=n_filters*2, kernel_size=3, batch_norm=batch_norm)

    u_u_0 = Conv2DTranspose(n_filters, (3,3), strides=(2,2), padding='same')(c_u_1)
    # skip layer 0
    cc_u_0 = concatenate([u_u_0, c_d_0], axis=3)
    d_u_0 = Dropout(dropout)(cc_u_0)
    c_u_0 = conv2d_block(d_u_0, n_filters=n_filters, kernel_size=3, batch_norm=batch_norm)
    
    output_segmentation = Conv2D(1, (1, 1), activation='sigmoid')(c_u_0)
    model = Model(inputs=[input_img], outputs=[output_segmentation])
    return model

In [ ]:
input_img = Input((nx, ny, 1), name='img')

model = create_unet(input_img, n_filters=16, dropout=0.025, batch_norm=True)
model.compile(optimizer=Adam(), loss=dice_coef_loss, metrics=["accuracy", f1])
model.summary()

In [ ]:
#!ls -ltr */models .\models\
!ls -ltr models/*.h5

In [ ]:
'models/'+source_string[0]+'-model-'+receiver_string+'-dice-fold-'+str(foldstring)+'.h5'

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.00001, verbose=1),
    ModelCheckpoint('models/'+source_string[0]+'-model-'+receiver_string+'-dice-fold-'+str(foldstring)+'.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
import time
t = time.time()
# do stuff
elapsed = time.time() - t
print('elapsed time',elapsed)

## Step 7. ```results = ``` cell to perform training

Horizontal DAS took 32 minutes (models/h-model-das-test.h5) : But only has 20 epochs, Michael recommends 50-100


In [ ]:
t = time.time()
results = model.fit(x_train, y_train, batch_size=1, epochs=20, callbacks=callbacks,
                   validation_data=(x_val, y_val))
elapsed = time.time() - t
print('elapsed time',elapsed)

In [ ]:


plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
#?model.evaluate
model.metrics_names

# `model.evaluate()` Returns...
    Scalar test loss (if the model has a single output and no metrics)
    or list of scalars (if the model has multiple outputs
    and/or metrics). The attribute `model.metrics_names` will give you
    the display labels for the scalar outputs.

In [ ]:
# Evaluate on validation data
model.evaluate(x_val, y_val, verbose=1)

In [ ]:
# Predict on train, val and test
preds_train = model.predict(x_train, verbose=1)
preds_val = model.predict(x_val, verbose=1)
preds_test = model.predict(x_test, verbose=1)

In [ ]:
# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
def plot_sample(X, y, preds, binary_preds, ix=None, filename=None):
    if ix is None:
        ix = random.randint(0, len(X))
        print('random slice is:',ix)

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(20, 10))
    im0 = ax[0,0].imshow(X[ix, ..., 0], cmap='gray')
    #if has_mask:
        #ax[0].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    fig.colorbar(im0, ax=ax[0,0], fraction=0.046, pad=0.04)
    ax[0,0].set_title('Seismic')

    im1 = ax[0,1].imshow(y[ix].squeeze())
    ax[0,1].set_title('Fault')
    fig.colorbar(im1, ax=ax[0,1], fraction=0.046, pad=0.04)
    
    im2 = ax[1,0].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    #if has_mask:
        #ax[2].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    fig.colorbar(im2, ax=ax[1,0], fraction=0.046, pad=0.04)
    ax[1,0].set_title('Fault Predicted')
    
    im3 = ax[1,1].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1)
    #if has_mask:
        #ax[3].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    fig.colorbar(im3, ax=ax[1,1], fraction=0.046, pad=0.04)
    ax[1,1].set_title('Fault Predicted (Binary)')
    fig.tight_layout();
    if filename!=None:
        plt.savefig(filename)

In [ ]:
# Check if training data looks all right
plot_sample(x_train, y_train, preds_train, preds_train_t, ix=1)

In [ ]:
# Check if valid data looks all right
plot_sample(x_val, y_val, preds_val, preds_val_t, ix=1)

In [ ]:
# Check if test data looks all right
plot_sample(x_test, y_test, preds_test, preds_test_t, ix=1, filename=source_string+receiver_string+'predictions_image.png')

In [ ]:
x_total = np.concatenate((x_train, x_val, x_test), axis=0)
y_total = np.concatenate((y_train, y_val, y_test), axis=0)
pred_total = np.concatenate((preds_train_t, preds_val_t, preds_test_t), axis=0)

In [ ]:
c_matrix = confusion_matrix(y_total.ravel(), pred_total.ravel())

In [ ]:
def print_roc_metrics(y_real, y_predict):

    c_matrix = confusion_matrix(y_real.ravel(), y_predict.ravel())
    f1 = f1_score(y_real.ravel(), y_predict.ravel())
    recall = recall_score(y_real.ravel(), y_predict.ravel())
    precision = precision_score(y_real.ravel(), y_predict.ravel())
    print("Confusion matrix:")
    print(c_matrix)
    print("F1 score: {:.4f}".format(f1))
    print("Recall score: {:.4f}".format(recall))
    print("Precision score: {:.4f}".format(precision))

In [ ]:
print_roc_metrics(y_test, preds_test_t)

''?confusion_matrix
Thus in binary classification, the count of 

true negatives is :math:`C_{0,0}`, 
false negatives is :math:`C_{1,0}`, 
true positives is :math:`C_{1,1}` and 
false positives is :math:`C_{0,1}`.''